In [1]:
!pip install pytorch_transformers

     |████████████████████████████████| 176 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 48.2 MB/s 
     |████████████████████████████████| 895 kB 39.8 MB/s 
     |████████████████████████████████| 131 kB 46.8 MB/s 
     |████████████████████████████████| 8.5 MB 40.5 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 138 kB 53.8 MB/s 
     |████████████████████████████████| 127 kB 51.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 123 kB 5.5 MB/s 


In [3]:
import torch
from pytorch_transformers import BertTokenizer, BertModel, BertForMaskedLM # Load BertTokenizer and the model 

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)




In [4]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
import copy
from torch.optim import lr_scheduler # learning rate scheduler module 
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from torch.utils.data import Dataset, DataLoader
from random import randrange
import torch.nn.functional as F

## Load Pre-trained model Tokenizer 

In [5]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # set the tokenizer 

INFO:pytorch_transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpldxs6chh
100%|██████████| 231508/231508 [00:00<00:00, 2147768.94B/s]
INFO:pytorch_transformers.file_utils:copying /tmp/tmpldxs6chh to cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_transformers.file_utils:creating metadata file for /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_transformers.file_utils:removing temp file /tmp/tmpldxs6chh
INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformer

In [6]:
text ='what is a pug'

In [7]:
tokenizer_result = tokenizer.tokenize(text)


In [8]:
tokenizer_result

['what', 'is', 'a', 'pu', '##g']

## Build Model

![대체 텍스트](./figures/bert_sentence.png)

![대체 텍스트](./figures/bert_inputs.png)

In [9]:

class BertForSequenceClassification(nn.Module):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    Params:
        `config`: a BertConfig class instance with the configuration to build a new model.
        `num_labels`: the number of classes for the classifier. Default = 2.
    Inputs:
        `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
            with the word token indices in the vocabulary. Items in the batch should begin with the special "CLS" token. (see the tokens preprocessing logic in the scripts
            `extract_features.py`, `run_classifier.py` and `run_squad.py`)
        `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token
            types indices selected in [0, 1]. Type 0 corresponds to a `sentence A` and type 1 corresponds to
            a `sentence B` token (see BERT paper for more details).
        `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
            selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
            input sequence length in the current batch. It's the mask that we typically use for attention when
            a batch has varying length sentences.
        `labels`: labels for the classification output: torch.LongTensor of shape [batch_size]
            with indices selected in [0, ..., num_labels].
    Outputs:
        if `labels` is not `None`:
            Outputs the CrossEntropy classification loss of the output with the labels.
        if `labels` is `None`:
            Outputs the classification logits of shape [batch_size, num_labels].
    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
    num_labels = 2
    model = BertForSequenceClassification(config, num_labels)
    logits = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, num_labels=2):
        super(BertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        nn.init.xavier_normal_(self.classifier.weight)
    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        return logits
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [10]:
from pytorch_pretrained_bert import BertConfig

config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

num_labels = 2
model = BertForSequenceClassification(num_labels)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(tokenizer_result)])

logits = model(tokens_tensor)

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpjh2iy0qi
100%|██████████| 433/433 [00:00<00:00, 413415.35B/s]
INFO:pytorch_transformers.file_utils:copying /tmp/tmpjh2iy0qi to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:pytorch_transformers.file_utils:creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:pytorch_transformers.file_utils:removing temp file /tmp/tmpjh2iy0qi
INFO:pytorch_transformers.modeling_utils:loading configuration fil

In [11]:
logits

tensor([[-0.5009, -0.4781]], grad_fn=<AddmmBackward0>)

## Load Dataset

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import os
import pandas as pd
data_path = '/content/drive/MyDrive/중진공 이어드림/1 20 nlp/실습2'

dat = pd.read_csv(os.path.join(data_path, 'Data/IMDB Dataset.csv'))

In [14]:
dat.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [15]:
review1 = tokenizer.tokenize(dat.review[1])
review3 = tokenizer.tokenize(dat.review[3])

[review1,review3 ]

[['a',
  'wonderful',
  'little',
  'production',
  '.',
  'the',
  'filming',
  'technique',
  'is',
  'very',
  'una',
  '##ss',
  '##uming',
  '-',
  'very',
  'old',
  '-',
  'time',
  '-',
  'bbc',
  'fashion',
  'and',
  'gives',
  'a',
  'comforting',
  ',',
  'and',
  'sometimes',
  'discomfort',
  '##ing',
  ',',
  'sense',
  'of',
  'realism',
  'to',
  'the',
  'entire',
  'piece',
  '.',
  'the',
  'actors',
  'are',
  'extremely',
  'well',
  'chosen',
  '-',
  'michael',
  'sheen',
  'not',
  'only',
  '"',
  'has',
  'got',
  'all',
  'the',
  'polar',
  '##i',
  '"',
  'but',
  'he',
  'has',
  'all',
  'the',
  'voices',
  'down',
  'pat',
  'too',
  '!',
  'you',
  'can',
  'truly',
  'see',
  'the',
  'seam',
  '##less',
  'editing',
  'guided',
  'by',
  'the',
  'references',
  'to',
  'williams',
  "'",
  'diary',
  'entries',
  ',',
  'not',
  'only',
  'is',
  'it',
  'well',
  'worth',
  'the',
  'watching',
  'but',
  'it',
  'is',
  'a',
  'terrific',
  '##ly

In [16]:
tokens_sentence1 = tokenizer.convert_tokens_to_ids(review1)
tokens_sentence3 = tokenizer.convert_tokens_to_ids(review3)

In [17]:
tokens_sentence1

[1037,
 6919,
 2210,
 2537,
 1012,
 1996,
 7467,
 6028,
 2003,
 2200,
 14477,
 4757,
 24270,
 1011,
 2200,
 2214,
 1011,
 2051,
 1011,
 4035,
 4827,
 1998,
 3957,
 1037,
 16334,
 1010,
 1998,
 2823,
 17964,
 2075,
 1010,
 3168,
 1997,
 15650,
 2000,
 1996,
 2972,
 3538,
 1012,
 1996,
 5889,
 2024,
 5186,
 2092,
 4217,
 1011,
 2745,
 20682,
 2025,
 2069,
 1000,
 2038,
 2288,
 2035,
 1996,
 11508,
 2072,
 1000,
 2021,
 2002,
 2038,
 2035,
 1996,
 5755,
 2091,
 6986,
 2205,
 999,
 2017,
 2064,
 5621,
 2156,
 1996,
 25180,
 3238,
 9260,
 8546,
 2011,
 1996,
 7604,
 2000,
 3766,
 1005,
 9708,
 10445,
 1010,
 2025,
 2069,
 2003,
 2009,
 2092,
 4276,
 1996,
 3666,
 2021,
 2009,
 2003,
 1037,
 27547,
 2135,
 2517,
 1998,
 2864,
 3538,
 1012,
 1037,
 3040,
 3993,
 2537,
 2055,
 2028,
 1997,
 1996,
 2307,
 3040,
 1005,
 1055,
 1997,
 4038,
 1998,
 2010,
 2166,
 1012,
 1996,
 15650,
 2428,
 3310,
 2188,
 2007,
 1996,
 2210,
 2477,
 1024,
 1996,
 5913,
 1997,
 1996,
 3457,
 2029,
 1010,
 2738,
 20

In [18]:
import torch.nn.functional as F

F.softmax(logits,dim=1)

tensor([[0.4943, 0.5057]], grad_fn=<SoftmaxBackward0>)

## Defined Data and DataLoader (IMDB)

In [19]:
# Splitting the data
from sklearn.model_selection import train_test_split
X = dat['review']
y = dat['sentiment']

X_train = dat['review'][:30684] 
y_train = dat['sentiment'][:30684]
X_test = dat['review'][30684:]
y_test = dat['sentiment'][30684:]


In [20]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

y_train = pd.get_dummies(y_train).values.tolist()
y_test = pd.get_dummies(y_test).values.tolist()


In [21]:
# Make the Dataset object for the dataloader 
max_seq_length = 256
class text_dataset(Dataset):
    def __init__(self,x_y_list, transform=None):
        
        self.x_y_list = x_y_list
        self.transform = transform
        
    def __getitem__(self,index):
        
        tokenized_review = tokenizer.tokenize(self.x_y_list[0][index]) # tokenize the review 
        
        if len(tokenized_review) > max_seq_length: # check the length of the sentence 
            tokenized_review = tokenized_review[:max_seq_length]
            
        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_review) # conver to ids 

        padding = [0] * (max_seq_length - len(ids_review)) # add pad token 
        
        ids_review += padding 
        
        assert len(ids_review) == max_seq_length
        
        #print(ids_review)
        ids_review = torch.tensor(ids_review)
        
        sentiment = self.x_y_list[1][index] # color        
        list_of_labels = [torch.from_numpy(np.array(sentiment))]
        
        
        return ids_review, list_of_labels[0]
    
    def __len__(self):
        return len(self.x_y_list[0])

In [22]:

batch_size = 16

train_lists = [X_train, y_train]
test_lists = [X_test, y_test]

training_dataset = text_dataset(x_y_list = train_lists )

test_dataset = text_dataset(x_y_list = test_lists )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [23]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            
            sentiment_corrects = 0
            
            
            # Iterate over data.
            for inputs, sentiment in dataloaders_dict[phase]:
                #inputs = inputs
                #print(len(inputs),type(inputs),inputs)
                #inputs = torch.from_numpy(np.array(inputs)).to(device) 
                inputs = inputs.to(device) 

                sentiment = sentiment.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    #print(inputs)
                    outputs = model(inputs)

                    outputs = F.softmax(outputs,dim=1)
                    
                    loss = criterion(outputs, torch.max(sentiment.float(), 1)[1])
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)

                
                sentiment_corrects += torch.sum(torch.max(outputs, 1)[1] == torch.max(sentiment, 1)[1])

                
            epoch_loss = running_loss / dataset_sizes[phase]

            
            sentiment_acc = sentiment_corrects.double() / dataset_sizes[phase]

            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} sentiment_acc: {:.4f}'.format(
                phase, sentiment_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'bert_model_test.pth')


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(float(best_loss)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [24]:
model.to(device)

#model.freeze_bert_encoder()
#model.classifier.weight.requires_grad = True
model

#    def freeze_bert_encoder(self):
#        for param in self.bert.parameters():
#            param.requires_grad = False
#    
#    def unfreeze_bert_encoder(self):
#        for param in self.bert.parameters():
#            param.requires_grad = True

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [25]:
lrlast = .001
lrmain = .00001
optim1 = optim.Adam(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
        {"params":model.classifier.parameters(), "lr": lrlast},
       
   ])

#optim1 = optim.Adam(model.parameters(), lr=0.001)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)


In [ ]:
model_ft1 = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)


starting
Epoch 0/9
----------


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


### Only Fine-tuning the last linear layer 

In [ ]:
lrlast = .001
lrmain = .00001
model.bert.eval() # eval mode (Bert)
optim1 = optim.Adam(
    [
        {"params":model.classifier.parameters(), "lr": lrlast},
       
   ])

#optim1 = optim.Adam(model.parameters(), lr=0.001)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)


In [ ]:
model_ft1 = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)
